#**COS30081 FNLP Tutorial 7: Question**

##**Lab Task Submission at bottom #PassTask6**
## **Name: Arthur Vincent Chin**
## **Student ID: 101218817**


**Pass Task Description:**

There are several forms of LSTM that can be used. At the very basic level a variation of using LSTM would be stack LSTM layers together. A sample code is given below on how it can be easily done using keras.

Now try to adapt either from  jupyter notebook 7b or 7c to stack 2, 3 or more layers togethers. Use the same datasets given in the notebook. Include your scripts here. Also Write down your observations (time, performance) [<50 words]

**Additional & Optional:**
You can try to implement other forms of LSTM such as GRU and bidirectional LSTM as well besides stacking.


In [1]:
import os
import re
import tarfile

import requests
!pip install pugnlp
from pugnlp.futil import path_status, find_files

/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')


In [2]:
# From the nlpia package for downloading data too big for the repo
import tqdm
BIG_URLS = {
    'w2v': (
        'https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz?dl=1',
        1647046227,
    ),
    'slang': (
        'https://www.dropbox.com/s/43c22018fbfzypd/slang.csv.gz?dl=1',
        117633024,
    ),
    'tweets': (
        'https://www.dropbox.com/s/5gpb43c494mc8p0/tweets.csv.gz?dl=1',
        311725313,
    ),
    'lsa_tweets': (
        'https://www.dropbox.com/s/rpjt0d060t4n1mr/lsa_tweets_5589798_2003588x200.tar.gz?dl=1',
        3112841563,  # 3112841312,
    ),
    'imdb': (
        'https://www.dropbox.com/s/yviic64qv84x73j/aclImdb_v1.tar.gz?dl=1',
        3112841563,  # 3112841312,
    ),
}

In [3]:
# These functions are part of the nlpia package which can be pip installed and run from there.
def dropbox_basename(url):
    filename = os.path.basename(url)
    match = re.findall(r'\?dl=[0-9]$', filename)
    if match:
        return filename[:-len(match[0])]
    return filename

def download_file(url, data_path='.', filename=None, size=None, chunk_size=4096, verbose=True):
    """Uses stream=True and a reasonable chunk size to be able to download large (GB) files over https"""
    if filename is None:
        filename = dropbox_basename(url)
    file_path = os.path.join(data_path, filename)
    if url.endswith('?dl=0'):
        url = url[:-1] + '1'  # noninteractive download
    if verbose:
        tqdm_prog = tqdm
        print('requesting URL: {}'.format(url))
    else:
        tqdm_prog = no_tqdm
    r = requests.get(url, stream=True, allow_redirects=True)
    size = r.headers.get('Content-Length', None) if size is None else size
    print('remote size: {}'.format(size))

    stat = path_status(file_path)
    print('local size: {}'.format(stat.get('size', None)))
    if stat['type'] == 'file' and stat['size'] == size:  # TODO: check md5 or get the right size of remote file
        r.close()
        return file_path

    print('Downloading to {}'.format(file_path))

    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            if chunk:  # filter out keep-alive chunks
                f.write(chunk)

    r.close()
    return file_path

def untar(fname):
    if fname.endswith("tar.gz"):
        with tarfile.open(fname) as tf:
            tf.extractall()
    else:
        print("Not a tar.gz file: {}".format(fname))

In [4]:
#  UNCOMMENT these 2 lines if you haven't already download the word2vec model and the imdb dataset
download_file(BIG_URLS['w2v'][0])
untar(download_file(BIG_URLS['imdb'][0]))

requesting URL: https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz?dl=1
remote size: 1647046227
local size: 1647046227
requesting URL: https://www.dropbox.com/s/yviic64qv84x73j/aclImdb_v1.tar.gz?dl=1
remote size: 84125825
local size: 84125825


In [5]:
import glob
import os

from random import shuffle

def pre_process_data(filepath):
    """
    This is dependent on your training data source but we will try to generalize it as best as possible.
    """
    positive_path = os.path.join(filepath, 'pos')
    negative_path = os.path.join(filepath, 'neg')
    
    pos_label = 1
    neg_label = 0
    
    dataset = []
    
    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
            
    for filename in glob.glob(os.path.join(negative_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((neg_label, f.read()))
    
    shuffle(dataset)
    
    return dataset

In [6]:
from nltk.tokenize import TreebankWordTokenizer
from gensim.models import KeyedVectors
word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=200000)

def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
    expected = []
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])

            except KeyError:
                pass  # No matching token in the Google w2v vocab
            
        vectorized_data.append(sample_vecs)

    return vectorized_data

In [7]:
def collect_expected(dataset):
    """ Peel of the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [8]:
def pad_trunc(data, maxlen):
    """ For a given dataset pad with zero vectors or truncate to maxlen """
    new_data = []

    # Create a vector of 0's the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
 
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [9]:
import numpy as np

dataset = pre_process_data('./aclImdb/train')
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

split_point = int(len(vectorized_data)*.2)
split_point2 = int(len(vectorized_data)*.25)

x_train = vectorized_data[:split_point]
y_train = expected[:split_point]
x_test = vectorized_data[split_point:split_point2]
y_test = expected[split_point:split_point2]

maxlen = 400
batch_size = 32         # How many samples to show the net before backpropogating the error and updating the weights
embedding_dims = 300    # Length of the token vectors we will create for passing into the Convnet
epochs = 20

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

##**Building LSTM Network (2 stacked layers)**

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

print('Build model...')
model = Sequential()

model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))

model.add(LSTM(num_neurons, return_sequences=True))
model.add(Dropout(.2))

model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 400, 50)           70200     
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 400, 50)           20200     
                                                                 
 dropout_1 (Dropout)         (None, 400, 50)           0         
                                                                 
 flatten (Flatten)           (None, 20000)             0         
                                                                 
 dense (Dense)               (None, 1)                 20001     
                                                                 
Total params: 110,401
Trainable params: 1

##**Training LSTM Network**

In [11]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/20
157/157 [==============================] - 84s 504ms/step - loss: 0.5839 - accuracy: 0.6894 - val_loss: 0.4471 - val_accuracy: 0.8136
Epoch 2/20
157/157 [==============================] - 77s 490ms/step - loss: 0.4575 - accuracy: 0.7940 - val_loss: 0.3667 - val_accuracy: 0.8416
Epoch 3/20
157/157 [==============================] - 77s 493ms/step - loss: 0.4034 - accuracy: 0.8264 - val_loss: 0.5280 - val_accuracy: 0.7416
Epoch 4/20
157/157 [==============================] - 78s 500ms/step - loss: 0.3523 - accuracy: 0.8500 - val_loss: 0.3637 - val_accuracy: 0.8480
Epoch 5/20
157/157 [==============================] - 79s 505ms/step - loss: 0.3079 - accuracy: 0.8716 - val_loss: 0.5340 - val_accuracy: 0.8048
Epoch 6/20
157/157 [==============================] - 79s 503ms/step - loss: 0.2697 - accuracy: 0.8856 - val_loss: 0.6645 - val_accuracy: 0.7424
Epoch 7/20
157/157 [==============================] - 78s 499ms/step - loss: 0.2250 - accuracy: 0.9084 - val_loss: 0.4862 - val_ac

##**Saving the LSTM Network**

In [12]:
model_structure = model.to_json()
with open("lstm_model1.json", "w") as json_file:
  json_file.write(model_structure)
model.save_weights("lstm_weights_stacked2.h5")

In [13]:
from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
  json_string = json_file.read()
model = model_from_json(json_string)
model.load_weights('lstm_weights1_stacked2.h5')

##**Prediction**

In [15]:
sample_1 = '''I'm hate that the dismal weather that had me down for so long, 
when will it break! Ugh, when does happiness return?  The sun is blinding and 
the puffy clouds are too thin.  I can't wait for the weekend.'''

# We pass a dummy value in the first element of the tuple just because our helper expects it from the way processed the initial data.  That value won't ever see the network, so it can be whatever.
vec_list = tokenize_and_vectorize([(1, sample_1)])

# Tokenize returns a list of the data (length 1 here)
test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

print("Sample's sentiment, 1 - pos, 2 - neg : {}".format(model.predict(test_vec)))
print("Raw output of sigmoid function: {}".format(model.predict(test_vec)))

Sample's sentiment, 1 - pos, 2 - neg : [[0.03730413]]
Raw output of sigmoid function: [[0.03730413]]


##**Findings**

####For 2 stacked layers, the LSTM network had 2 stacked layers with 50 neurons and 2 dropout layers with a rate of 0.2. Without any other further fine tuning other than adding the new layers, the training took 22 minutes and 44 seconds to complete and it finished training with a training accuracy of 99.06% and a loss of 0.0258.

##**Building LSTM Network (3 stacked layers)**

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons_2 = 50

print('Build model...')
model2 = Sequential()

model2.add(LSTM(num_neurons_2, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model2.add(Dropout(.2))

model2.add(LSTM(num_neurons_2, return_sequences=True)) # 2nd stacked layer
model2.add(Dropout(.2))

model2.add(LSTM(num_neurons_2, return_sequences=True)) # 3rd stacked layer
model2.add(Dropout(.2))

model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

model2.compile('rmsprop', 'binary_crossentropy',  metrics=['accuracy'])
print(model2.summary())

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 400, 50)           70200     
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 400, 50)           20200     
                                                                 
 dropout_1 (Dropout)         (None, 400, 50)           0         
                                                                 
 lstm_2 (LSTM)               (None, 400, 50)           20200     
                                                                 
 dropout_2 (Dropout)         (None, 400, 50)           0         
                                                                 
 flatten (Flatten)           (None, 20000

##**Training LSTM Network**

In [11]:
model2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/20
157/157 [==============================] - 123s 743ms/step - loss: 0.6150 - accuracy: 0.6714 - val_loss: 0.7204 - val_accuracy: 0.6016
Epoch 2/20
157/157 [==============================] - 114s 728ms/step - loss: 0.4906 - accuracy: 0.7794 - val_loss: 0.6395 - val_accuracy: 0.6936
Epoch 3/20
157/157 [==============================] - 117s 743ms/step - loss: 0.4244 - accuracy: 0.8174 - val_loss: 0.6425 - val_accuracy: 0.7224
Epoch 4/20
157/157 [==============================] - 116s 741ms/step - loss: 0.3817 - accuracy: 0.8368 - val_loss: 0.5850 - val_accuracy: 0.7376
Epoch 5/20
157/157 [==============================] - 116s 742ms/step - loss: 0.3445 - accuracy: 0.8502 - val_loss: 0.5199 - val_accuracy: 0.7872
Epoch 6/20
157/157 [==============================] - 117s 748ms/step - loss: 0.3057 - accuracy: 0.8748 - val_loss: 0.4380 - val_accuracy: 0.8152
Epoch 7/20
157/157 [==============================] - 116s 739ms/step - loss: 0.2746 - accuracy: 0.8862 - val_loss: 0.7065 -

##**Saving the LSTM Network**

In [14]:
model_structure = model2.to_json()
with open("lstm_model1.json", "w") as json_file:
  json_file.write(model_structure)
model2.save_weights("lstm_weights_stacked3.h5")

In [15]:
from keras.models import model_from_json
with open("lstm_model1.json", "r") as json_file:
  json_string = json_file.read()
model2 = model_from_json(json_string)
model2.load_weights('lstm_weights_stacked3.h5')

##**Prediction**

In [16]:
sample_1 = '''I'm hate that the dismal weather that had me down for so long, 
when will it break! Ugh, when does happiness return?  The sun is blinding and 
the puffy clouds are too thin.  I can't wait for the weekend.'''

# We pass a dummy value in the first element of the tuple just because our helper expects it from the way processed the initial data.  That value won't ever see the network, so it can be whatever.
vec_list = tokenize_and_vectorize([(1, sample_1)])

# Tokenize returns a list of the data (length 1 here)
test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

print("Sample's sentiment, 1 - pos, 2 - neg : {}".format(model2.predict(test_vec)))
print("Raw output of sigmoid function: {}".format(model2.predict(test_vec)))

Sample's sentiment, 1 - pos, 2 - neg : [[0.94795674]]
Raw output of sigmoid function: [[0.94795674]]


##**Findings**

####For 3 stacked layers, the LSTM network had 3 stacked layers with 50 neurons and 3 dropout layers with a rate of 0.2. Without any other further fine tuning other than adding the new layers, the training took 39 minutes to complete and it finished training with a training accuracy of 98% and a loss of 0.04. This model did well but not as good as the 2 stacked layers model before it.